In [1]:
from llamaapi import LlamaAPI
from dotenv import load_dotenv
import os

load_dotenv()
LLAMAAPI_KEY = os.getenv('LLAMAAPI_KEY')

llama = LlamaAPI(LLAMAAPI_KEY)

In [2]:
import pandas as pd
from openpyxl import load_workbook

def xlsx2df(filename):
    wb = load_workbook(filename, data_only=False)
    sheet = wb["Sheet1"]

    data = []
    for row in sheet.iter_rows():
        row_data = []
        for cell in row:
            row_data.append(cell.value)
        data.append(row_data)

    df = pd.DataFrame(data)
    return df

# df = xlsx2df("DCF model.xlsx")
df = xlsx2df("Actuarial Calculation.xlsx")
print(df)


                          0          1
0               Sum Assured   100000.0
1            Annual Premium     1000.0
2            Mortality Rate       0.01
3             Interest Rate       0.03
4  Expected Benefit Payment     =B1*B3
5           Discount Factor  =1/(1+B4)
6    PV of Expected Benefit     =B5*B6
7       Net Premium Reserve     =B7-B2


In [15]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_experimental.llms import ChatLlamaAPI
from langchain_openai import ChatOpenAI

model_llama = ChatLlamaAPI(client=llama, model="llama3.1-8b", temperature=0, verbose=True)
model = ChatOpenAI(model="gpt-4o", temperature=0, verbose=True)

agent = create_pandas_dataframe_agent(model, df, allow_dangerous_code=True, number_of_head_rows=10)

In [24]:
from langchain.globals import set_debug
set_debug(False)

response = agent.invoke("This dataframe contains cells from an Excel spreadsheet. Some of these cells contain data labels, some contain data and others contain formulas. Convert this dataframe to Python code such that data labels are variable names, data are variable values and formulas use the variable names. Convert all the cells and formulas. Note that the first row index in Excel is 1.")
print(response['output'])

```python
Sum_Assured = 100000.0
Annual_Premium = 1000.0
Mortality_Rate = 0.01
Interest_Rate = 0.03
Expected_Benefit_Payment = Sum_Assured * Mortality_Rate
Discount_Factor = 1 / (1 + Interest_Rate)
PV_of_Expected_Benefit = Expected_Benefit_Payment * Discount_Factor
Net_Premium_Reserve = PV_of_Expected_Benefit - Annual_Premium
```
